# CS280 Programming Assignment 1
Naive Bayes Spam Filter
<br>
Dataset: TREC06 Corpus<br>
    train: 000 to 070<br>
    test: 071 to 126<br>

## 2.1 Classifier Construction and Evaluation
1. Parse the documents in the training set

In [42]:
import os
import re

data_path = 'data'

V = []
for root, dirs, files in os.walk(data_path+'\\000', topdown=False):
    for filename in files:
        with open(os.path.join(root, filename), 'r', errors='ignore') as file:
            for line in file:
                pattern = re.compile(r'\s+(?=[a-zA-Z^.])')
                words = pattern.split(line)
                print(words)
                for word in words:
                    if word not in V:
                        V.append(word.strip())


['Received:', 'from', 'rodan.UU.NET', 'by', 'aramis.rutgers.edu (5.59/SMI4.0/RU1.4/3.08) \n']
['', 'id', 'AA23563;', 'Mon, 27', 'Jul 92 22:28:01', 'EDT\n']
['Received:', 'from', 'relay2.UU.NET', 'by', 'rodan.UU.NET', 'with', 'SMTP \n']
['\t(5.61/UUNET-mail-drop)', 'id', 'AA06229;', 'Mon, 27', 'Jul 92 22:27:59 -0400\n']
['Received:', 'from', 'uunet.uu.net (via', 'LOCALHOST.UU.NET)', 'by', 'relay2.UU.NET', 'with', 'SMTP \n']
['\t(5.61/UUNET-internet-primary)', 'id', 'AA26262;', 'Mon, 27', 'Jul 92 22:28:08 -0400\n']
['Received:', 'from', 'sarto.UUCP', 'by', 'uunet.uu.net', 'with', 'UUCP/RMAIL\n']
['\t(queueing-rmail)', 'id 222745.1653;', 'Mon, 27', 'Jul 1992 22:27:45', 'EDT\n']
['Newsgroups:', 'soc.religion.christian\n']
['Path:', 'jhpb\n']
['From:', 'jhpb@sarto.budd-lake.nj.us (Joseph', 'H.', 'Buehler)\n']
['Subject:', 'new', 'Catholic', 'mailing', 'list', 'now', 'up', 'and', 'running\n']
['Message-Id: <JHPB.92Jul27221355@sarto.budd-lake.nj.us>\n']
['Sender:', 'jhpb@sarto.budd-lake.nj.us

['', 'for <lawanda@groucho.cs.psu.edu>;', 'Thu, 22', 'Apr 1993 04:44:56 -0800\n']
['Message-ID: <350t542n.9107213@embroidme.com>\n']
['DomainKey-Signature:', 'a=rsa-sha1;', 'q=dns;', 'c=nofws;', 's=dystrophy.octennial;', 'd=embroidme.com;\n']
['', 'b=E7z0w4TJ51U7gUboynBcOgYU4YT3P4JHDQrBDjd91w9VvUDIFkqx2qQM2HZWtlF4QoXnsSUVqCjCBFJG;\n']
['Date:', 'Thu, 22', 'Apr 1993 04:44:56 -0800\n']
['From: "Mr', 'Kali', 'Lydon" <xmfulrbs@1internet.us>\n']
['To:', 'gina@groucho.cs.psu.edu\n']
['Subject:', 'we', 'will', 'not', 'disappoint', 'you\n']
['Content-return:', 'allowed\n']
['X-Mailer:', 'devMail.Net (3.2.2205.6680-2)\n']
['X-Authentication-Warning:', 'localhost.localdomain:', 'apache', 'set', 'sender', 'to', 'xmfulrbs@1internet.us', 'using -f\n']
['X-Virus-Scanned:', 'amavisd-new', 'at', 'embroidme.com\n']
['Mime-Version: 1.0\n']
['Content-Type:', 'multipart/alternative;\n']
['', 'boundary="------------002_Dragon106291842498"\n']
['\n']
['This', 'is', 'a', 'multi-part', 'message', 'in', 'MIME'

['\n']
['-rob\n']
['\n']
['Received:', 'from', 'netx-61-161-hyd.netexcell.com (HELO', 'top.maxi4u.com) ([203.196.161.61])\n']
['', 'by', 'groucho.cs.psu.edu', 'with', 'SMTP', 'id <2583>;', 'Sat, 24', 'Apr 1993 21:28:37 -0400\n']
['Message-ID: <614601c67616$629c8220$007fc410@lerarnnarxernar>\n']
['From: "Aaron" <lerarnnarxernar@top.maxi4u.com>\n']
['To: "Alton" <amie@groucho.cs.psu.edu>\n']
['Cc: <juliet@groucho.cs.psu.edu>,\n']
['\t<jimmie@groucho.cs.psu.edu>,\n']
['\t<reva@groucho.cs.psu.edu>,\n']
['\t<rosario@groucho.cs.psu.edu>,\n']
['\t<katina@groucho.cs.psu.edu>,\n']
['\t<mike@groucho.cs.psu.edu>,\n']
['\t<kelley@groucho.cs.psu.edu>\n']
['Subject:', 'decide', 'yet\n']
['Date:', 'Sat, 24', 'Apr 1993 22:04:32 +0100\n']
['MIME-Version: 1.0\n']
['Content-Type:', 'text/plain;\n']
['', 'charset="us-ascii"\n']
['Content-Transfer-Encoding: 7bit\n']
['X-Priority: 3\n']
['X-MSMail-Priority:', 'Normal\n']
['X-Mailer:', 'Microsoft', 'Outlook', 'Express 6.00.2600.0000  \n']
['X-MimeOLE:', 'Pro

['<DIV\n']
['\n']
['style=3D"\n']
['\n']
['FLOAT\n']
['\n']
['\n']
[':\n']
['left\n']
['"> <DIV>', 'n</DIV> <DIV> <STRONG>', 'L</STRONG></DIV> <DIV> <STRONG> =\n']
['A</STRONG></DIV> <DIV>', 'v</DIV> <DIV>', 'o</DIV> <DIV>', 'b</DIV> <DIV> =\n']
['<STRONG>', 'A</STRONG></DIV> </DIV>\n']
['<DIV\n']
['\n']
['style=3D"\n']
['\n']
['FLOAT\n']
['\n']
['\n']
[':\n']
['left\n']
['"> <DIV>', 'a</DIV> <DIV> <STRONG>', 'I</STRONG></DIV> <DIV> <STRONG> =\n']
['G</STRONG></DIV> <DIV>', 'i</DIV> <DIV>', 'z</DIV> <DIV>', 'i</DIV> <DIV> =\n']
['<STRONG>', 'L</STRONG></DIV> </DIV>\n']
['<DIV\n']
['\n']
['style=3D"\n']
['\n']
['FLOAT\n']
['\n']
['\n']
[':\n']
['left\n']
['"> <DIV>', 'x</DIV> <DIV> <STRONG>', 'U</STRONG></DIV> <DIV> <STRONG> =\n']
['R</STRONG></DIV> <DIV>', 't</DIV> <DIV>', 'a</DIV> <DIV>', 'e</DIV> <DIV> =\n']
['<STRONG>', 'I</STRONG></DIV> </DIV>\n']
['<DIV\n']
['\n']
['style=3D"\n']
['\n']
['FLOAT\n']
['\n']
['\n']
[':\n']
['left\n']
['"> <DIV> <BR></DIV> <DIV> <STRONG>', 'M</STRONG>

['size=3D"2"><strong>=CC=E5=F1=F2=EE =F1=EB=F3=E6=E1=FB =\n']
['=E4=E5=EB=EE=EF=F0=EE=E8=E7=E2=EE=E4=F1=F2=E2=E0 =E2 =\n']
['=F1=E8=F1=F2=E5=EC=E5 =F3=EF=F0=E0=E2=EB=E5=ED=E8=FF, =\n']
['=EE=F1=ED=EE=E2=ED=FB=E5 =E7=E0=E4=E0=F7=E8, =F4=F3=ED=EA=F6=E8=E8, =\n']
['=EF=F0=E0=E2=E0.</strong> =CD=EE=F0=EC=E0=F2=E8=E2=FB =\n']
['=F7=E8=F1=EB=E5=ED=ED=EE=F1=F2=E8, =EE=F1=EE=E1=E5=ED=ED=EE=F1=F2=E8 =\n']
['=E2=E7=E0=E8=EC=EE=E4=E5=E9=F1=F2=E2=E8=FF =F1=EB=F3=E6=E1=FB =C4=CE=D3 =\n']
['=F1 =E4=EE=EB=E6=ED=EE=F1=F2=ED=FB=EC=E8 =EB=E8=F6=E0=EC=E8 =E8 =\n']
['=E8=ED=FB=EC=E8 =F1=F2=F0=F3=EA=F2=F3=F0=ED=FB=EC=E8 =\n']
['=EF=EE=E4=F0=E0=E7=E4=E5=EB=E5=ED=E8=FF=EC=E8.</font></div>\n']
['<div><strong><font', 'size=3D"2">3.</font></strong><font =\n']
['size=3D"2"><strong>=CE=F0=E3=E0=ED=E8=E7=E0=F6=E8=FF =\n']
['=FD=F4=F4=E5=EA=F2=E8=E2=ED=EE=E3=EE =\n']
['=E4=EE=EA=F3=EC=E5=ED=F2=EE=EE=E1=EE=F0=EE=F2=E0. =\n']
['</strong>&nbsp;=C2=E8=E4=FB =E1=EB=E0=ED=EA=EE=E2; =\n']
['=F0=E5=EA=E2=E8=E7=E8=F2=FB, =E

...

['\n']
['\n']
['\x1b$BA4$F$N=P2q$$$O%3%3$K$"$k!*\x1b(B\n']
['\x1b$BA49q$NCK=w$,5a$a$k=P2q$$7O$NI,?\\;v9`$r=87k$$$?$7$^$7$?!#$-$C$H$4K~B-$$$?$@$1$^$9!*\x1b(B\n']
['\x1b$B%.%c%k7O;(;o!&%l%G%#%3%_$K$bBgNL$K9-9p7G:\\Cf$J$N$G!"=w@-2q0w$b%.%c%k$r%a%$%s$K$?$/$5$s$N%l%G%#$,5.J}$r$*BT$A$G$9!#\x1b(B\n']
['http://www.yamabari.com/?num=3349\n']
['\n']
['\x1b$B!!!!!z8r49$OK\\F|3+;O!z\x1b(B\n']
['\n']
["\x1b$BB~:#$h$j?75,EPO?<T$N3'MMA40w$K\x1b(B\n"]
['\x1b$B9b3[%"%U%?!<\x1b(BVIP\x1b$B=w@-$r\x1b(B100\x1b$B!sCg2p$$$?$7$^$9!#\x1b(B\n']
['\x1b$B"($*#1?MMM\x1b(B/\x1b$B#2!A#4L>>R2p\x1b(B\n']
['http://www.yamabari.com/?num=3349\n']
['\n']
['\x1b$BEPO?$7$F$+$i8^J,0JFbI,$:O"Mm$,Mh$k!*\x1b(B\n']
['\x1b$BEPO?L5NA$J$N$G!"$40B?4$GMxMQ$7$F$/$@$5$$!*!#\x1b(B\n']
["\x1b$BDI?-!'=w@-2q0w$G$*6b$KIT<+M3$JJ}$O$$$^$;$s!*\x1b(B\n"]
['\x1b$B!!!!\x1b(B\n']
['\n']
['\n']
['\n']
['\n']
['\n']
['\x1b$B8r495qH]\x1b(Bnomail@mel-ch.com\n']
['\n']
['\n']
['\n']
['\n']
['Received:', 'from', 'psuvax1.cse.psu.edu ([130.203.2.4])', 'b

['+', 'else\n']
['+', 'ret =', 'unlink(f->path);\n']
['+', 'if (ret)\n']
['+', 'err =', 'sys_errlist[errno];\n']
['+ \t\t\t}\n']
['', 'free(fd);\n']
['  \t\t}\n']
['***************\n']
['*** 760,764 ****\n']
['  \t}\n']
['', 'f->qid.path =', 'qid(&stbuf);\n']
['!', 'f->qid.vers = 0;\n']
['', 'f->stbuf =', 'stbuf;\n']
['', 'return 0;\n']
['--- 811,815 ----\n']
['  \t}\n']
['', 'f->qid.path =', 'qid(&stbuf);\n']
['!', 'f->qid.vers =', 'stbuf.st_mtime;\n']
['', 'f->stbuf =', 'stbuf;\n']
['', 'return 0;\n']
['\n']
['Received:', 'from', 'psuvax1.cse.psu.edu ([130.203.2.4])', 'by', 'groucho.cse.psu.edu', 'with', 'SMTP', 'id <2542>;', 'Wed, 3', 'Nov 1993 13:45:01 -0500\n']
['Received:', 'from', 'skinner.cs.uoregon.edu ([128.223.4.13])', 'by', 'psuvax1.cse.psu.edu', 'with', 'SMTP', 'id <292358>;', 'Wed, 3', 'Nov 1993 13:26:23 -0500\n']
['Received:', 'from', 'majestix.cs.uoregon.edu', 'by', 'skinner.cs.uoregon.edu', 'with', 'SMTP', 'id', 'AA25787\n']
['  (5.65/IDA-1.4.2', 'for 9fans@cse.psu.edu

[' \n']
['for', 'the', 'full', 'details', 'read', 'from', 'any', 'stock', 'reporting', 'site. \n']
[' \n']
['The', 'key', 'is,', 'knowing', 'when', 'to', 'get', 'on', 'and', 'when', 'to', 'get', 'off', 'a', 'stock,', 'for  \n']
['successful', 'day', 'trading.', 'CWTD', 'has', 'distinct', 'patterns', 'to', 'watch', 'for. \n']
[' \n']
['Yesterday', 'CWTD', 'volume', 'spiked', 'again 53%.', 'Today', 'we', 'got', 'word', 'that', 'there  \n']
['will', 'be', 'more', 'exciting', 'news', 'early', 'next', 'week.', 'We', 'are', 'still', 'hovering', 'at  \n']
['$1.98,', 'up 48', 'cents', 'since', 'this', 'climb', 'began. \n']
[' \n']
['Take', 'advantage', 'of', 'the', 'current', 'price', 'on', 'Monday,', 'we', 'expect', 'to', 'see', 'it', 'begin  \n']
['climbing', 'Monday', 'and', 'Tuesday', 'in', 'anticipation', 'of', 'the', 'coming', 'news', 'that', 'stirred  \n']
['the', 'market', 'last', 'week. \n']
[' \n']
['This', 'ride', 'is', 'not', 'over.', 'Jump', 'on', 'now', 'and', 'ride', 'the', 'pri

['conformity', 'with', 'the', 'latest', 'transfer', 'regulation', 'signed', 'by', 'the', 'Organization /\n']
['Netherlands', 'Government', 'since', 'this', 'is', 'part', 'of', 'our', 'Security', 'protocol', 'measures\n']
['to', 'avoid', 'double', 'claiming', 'and', 'unwarranted', 'abuse', 'of', 'this', 'program', 'by', 'some\n']
['participants \n']
['\n']
['We', 'also', 'wish', 'to', 'bring', 'to', 'your', 'notice', 'our', 'end', 'of', 'year', 'premium', 'stakes', 'draw\n']
['where', 'you', 'stand', 'a', 'chance', 'of', 'winning', 'up', 'to 25', 'Million', 'Euros ;', 'we', 'hope', 'that\n']
['with', 'a', 'part', 'of', 'your', 'prize', 'you', 'will', 'participate', 'in', 'it.\n']
['\n']
['To', 'file', 'for', 'your', 'claim,', 'please', 'contact', 'your', 'fiducially', 'agent\n']
['', 'Mr.', 'Paul', 'Joanson\n']
['Of', 'The', 'Euro', 'Accredited', 'Payment', 'Agent', 'Immediately \n']
['\n']
['TEL: +31 627 200 037 \n']
['\n']
['FAX: +31 847 180 870\n']
['\n']
['Email:', 'pauljoanson@nets

['', 'with', 'SMTP', 'id <2562>;', 'Sat, 13', 'Nov 1993 20:55:26 -0500\n']
['Received:', 'from 30.193.16.68', 'by ;', 'Sun, 14', 'Nov 1993 00:55:02 +0400\n']
['Message-ID: <CMJJPSPPWCLJWOFJCQPPC@yahoo.com>\n']
['From: "ÀÌÇöÈñ" <9tkt00t0kdf0dd3d@daum.net>\n']
['Reply-To: "¹ÚÇöÈñ" <9tkt00t0kdf0d3dd@daum.net>\n']
['To:', 'karina@groucho.cse.psu.edu\n']
['Subject: ++++++++++++++ ¾î·Á¿ì½ÃÁÒ? ÀúÈñ°¡ µµ¿Í µå¸®°Ú½À´Ï´Ù. ¹«ÀÚ°Ý ¤§ ¤À;Ãâ....\n']
['Date:', 'Sat, 13', 'Nov 1993 13:55:02 -0700\n']
['X-Mailer:', 'Internet', 'Mail', 'Service (5.5.2650.21)\n']
['MIME-Version: 1.0\n']
['Content-Type:', 'text/html\n']
['<html>\n']
['<head>\n']
['<meta', 'http-equiv="Content-Type"', 'content="text/html;', 'charset=euc-kr">\n']
['<title></title>\n']
['<style', 'type="text/css">\n']
['<!--\n']
['body {\n']
['', 'margin-left: 0px;\n']
['', 'margin-top: 0px;\n']
['', 'margin-right: 0px;\n']
['', 'margin-bottom: 0px;\n']
['}\n']
['-->\n']
['</style></head>\n']
['\n']
['<body>\n']
['<table', 'width="650"', 'bo

In [43]:
V

['Received:',
 'from',
 'rodan.UU.NET',
 'by',
 'aramis.rutgers.edu (5.59/SMI4.0/RU1.4/3.08)',
 '',
 'id',
 'AA23563;',
 'Mon, 27',
 'Jul 92 22:28:01',
 'EDT',
 'relay2.UU.NET',
 'with',
 'SMTP',
 '(5.61/UUNET-mail-drop)',
 'AA06229;',
 'Jul 92 22:27:59 -0400',
 'uunet.uu.net (via',
 'LOCALHOST.UU.NET)',
 'SMTP',
 '(5.61/UUNET-internet-primary)',
 'AA26262;',
 'Jul 92 22:28:08 -0400',
 'sarto.UUCP',
 'uunet.uu.net',
 'UUCP/RMAIL',
 '(queueing-rmail)',
 'id 222745.1653;',
 'Jul 1992 22:27:45',
 'EDT',
 'Newsgroups:',
 'soc.religion.christian',
 'Path:',
 'jhpb',
 'From:',
 'jhpb@sarto.budd-lake.nj.us (Joseph',
 'H.',
 'Buehler)',
 'Subject:',
 'new',
 'Catholic',
 'mailing',
 'list',
 'now',
 'up',
 'and',
 'running',
 'Message-Id: <JHPB.92Jul27221355@sarto.budd-lake.nj.us>',
 'Sender:',
 'H',
 'Buehler)',
 'Organization:',
 'none',
 'Date:',
 'Tue, 28',
 'Jul 1992 03:13:55',
 'GMT',
 'Content-Type:',
 'text',
 'Content-Length: 2745',
 'Apparently-To: <soc-religion-christian>',
 '',
 'T

In [2]:
labels_file_path = 'trec06p-cs280\labels'

labels_file = open(labels_file_path)
labels_file.close()